In [1]:
from ipynb.fs.full.setup import *

6
5
6
5
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
class A(index, displacement):

    val = "value that stays in class without changning with instance"

    def __init__(self, xy):

        if xy == "x":

            if transient:
                self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy
            else:
                self.a_P = ((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy

            self.a_N = + A.coef(xy, "N", "u")*(Sfy)/dy
            self.a_S = - A.coef(xy, "S", "u")*(Sfy)/dy
            self.a_E = + A.coef(xy, "E", "u")*(Sfx)/dx
            self.a_W = - A.coef(xy, "W", "u")*(Sfx)/dx

        if xy == "y":

            if transient:
                self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 
            else:
                self.a_P = ((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 

            self.a_N = + A.coef(xy, "N", "v")*(Sfy)/dy
            self.a_S = - A.coef(xy, "S", "v")*(Sfy)/dy
            self.a_E = + A.coef(xy, "E", "v")*(Sfx)/dx
            self.a_W = - A.coef(xy, "W", "v")*(Sfx)/dx

    def createMatrix(self):

        A_no_boundary = np.zeros([ny*nx, ny*nx])

        for k in range(0, ny*nx):
            if cell_index().no_boundary(k):      
                    A_no_boundary[k, k] = self.a_P
                    A_no_boundary[k, index(k).n] = -self.a_N
                    A_no_boundary[k, index(k).s] = -self.a_S
                    A_no_boundary[k, index(k).e] = -self.a_E
                    A_no_boundary[k, index(k).w] = -self.a_W

        return A_no_boundary

    # b temporal term
    def b_temp(U_old, U_old_old, k, xy):

        if xy == "x":
            if transient:
                b_temporal_term = (rho/(dt**2))*( 2*(U_old[k,0])*dx*dy - U_old_old[k,0]*dx*dy)
            else:
                b_temporal_term = 0

        if xy == "y":
            if transient:
                b_temporal_term = (rho/(dt**2))*( 2*(U_old[k,1])*dx*dy - U_old_old[k,1]*dx*dy)
            else:
                b_temporal_term = 0
        
        return b_temporal_term
    
    def b_diff(k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"

        b_diffusion = (
                                Sfy*A.coef(xy, "N", uv)*(
                                    (A.corner("NE", uv, U_previous, k) - A.corner("NW", uv, U_previous, k))
                                    /dx)
                                + 
                                Sfy*A.coef(xy, "S", uv)*(
                                    (A.corner("SE", uv, U_previous, k) - A.corner("SW", uv, U_previous, k))
                                    /dx)
                                +
                                Sfx*A.coef(xy, "E", uv)*(
                                    (A.corner("NE", uv, U_previous, k) - A.corner("SE", uv, U_previous, k))
                                    /dy)
                                +
                                Sfx*A.coef(xy, "W", uv)*(
                                    (A.corner("NW", uv, U_previous, k) - A.corner("SW", uv, U_previous, k))
                                    /dy)
        )

        return b_diffusion
    
    def corner(corner_placement, uv, U_previous, k):
        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)

        if corner_placement == "NE":
            return (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            return (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            return (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            return (1/4)*(disp.P + disp.NW + disp.N + disp.W)


    
    def coef(xy, face, uv):
        # could have and statements
        if xy == "x":
            if face == "N":
                if uv == "u":
                    return mu
                if uv == "v":
                    return mu
            if face == "S":
                if uv == "u":
                    return - mu
                if uv == "v":
                    return - mu
            if face == "E":
                if uv == "u":
                    return 2*mu + Lambda
                if uv == "v":
                    return Lambda
            if face == "W":
                if uv == "u":
                    return - (2*mu + Lambda)
                if uv == "v":
                    return - Lambda

        elif xy == "y":
            if face == "N":
                if uv == "u":
                    return Lambda
                if uv == "v":
                    return 2*mu + Lambda
            if face == "S":
                if uv == "u":
                    return - Lambda
                if uv == "v":
                    return - (2*mu + Lambda)
            if face == "E":
                if uv == "u":
                    return mu
                if uv == "v":
                    return mu
            if face == "W":
                if uv == "u":
                    return - mu
                if uv == "v":
                    return - mu

U_previous = U_new

